In [ ]:
# default_exp core

In [ ]:
#export
from urllib.request import urlopen
from bs4 import BeautifulSoup
import statistics as st

# intel_invest

> A package to obtain the relevant figures for a company and apply some rules of thumb precribed in the book Intelligent Investor, by Benjamin Graham.

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
type(float("NaN"))

float

In [ ]:
#export 
def _flotify(n: str) -> float:
    try:
        return float(n)
    except ValueError:
        return float("NaN")

In [ ]:
# export
class Stock:
    def __init__(self, symbol):
        self.symbol = symbol
        self.url = f"http://www.marketwatch.com/investing/stock/{symbol}/financials"

    def _get_soup(self) -> str:
        return BeautifulSoup(urlopen(self.url).read(), features="html.parser")

    def get_price(self) -> float:
        """Fetchs the price of stock."""
        txt = self._get_soup()
        price = txt.find("bg-quote", {"class": "value"})
        return _flotify(price.text)

    def get_esp(self) -> list:
        """Fetches earning per share or last 5 years."""
        txt = self._get_soup()
        titles = txt.findAll("td", {"class": "overflow__cell fixed--column"})
        esp_title = [t for t in titles if "EPS (Basic)" in t.text]
        if esp_title:
            esp_title = esp_title[0]
        eps = [
            _flotify(t.text)
            for t in esp_title.findNextSiblings(attrs={"class": "overflow__cell"})[:5]
        ]
        return eps
    
    def max_price(self) -> float:
        """Calculates the maximum price one should pay for a stock.
        
        Max stock price = 25 * 5 year average EPS.
        """
        return 25 * st.mean(self.get_esp())